In [0]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from pyspark.sql import SQLContext
from pyspark import SparkContext
from pyspark.mllib.evaluation import RegressionMetrics

In [0]:
%sh 
curl -O https://s3.amazonaws.com/cs6350.0u1/Project/infy_stock.csv

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed

 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0
100 30177 100 30177 0 0 69692 0 --:--:-- --:--:-- --:--:-- 69692

In [0]:
inputData = sqlContext.read.format("csv").option("header", "true").option("inferSchema", "true").load("file:/databricks/driver/infy_stock.csv")

In [0]:
df = inputData.drop(*['Date','Symbol','Series'])

In [0]:
#Params that can be changed 
trainSplit = 0.9
timeSteps = 20
lstmUnits = 100
dropoutRate = 0.4
modelOptimizer = 'Adam' #SGD, RMSprop, Adam, Adadelta, Adagrad, Adamax, Nadam, Ftrl
modelLoss ='mean_squared_error' #mean_squared_error, poisson, mean_squared_logarithmic_error
numEpochs = 100

In [0]:
trainSize = int(df.count() * trainSplit)
testSize = df.count()- trainSize
print(trainSize, testSize)

223 25

In [0]:
train = sqlContext.createDataFrame(df.head(trainSize), df.schema)
test = df.subtract(train)

In [0]:
train = train.toPandas()
test = test.toPandas()

In [0]:
scaler = MinMaxScaler(feature_range=(0,1))
scaleColumns = ['Prev Close', 'Open', 'High', 'Low', 'Last', 'VWAP', 'Volume', 'Turnover','Trades' ,'Deliverable Volume', '%Deliverble']
predictColumn = ['Close']
train[scaleColumns] = scaler.fit_transform(train.loc[:, scaleColumns])
test[scaleColumns] = scaler.fit_transform(test.loc[:, scaleColumns])

closeScaler =MinMaxScaler(feature_range=(0,1))
train[predictColumn] = closeScaler.fit_transform(train.loc[:, predictColumn])
test[predictColumn] = closeScaler.fit_transform(test.loc[:, predictColumn])

In [0]:
def generateSeqs(input, target, steps):
  inputData = []
  targetData = []
  for x in range (len(input) - steps):
    #get series of data
    inputData.append(input.iloc[x: (x + steps)].values)
    #get actual value of Close and append for target value
    targetData.append(target.iloc[x + steps])
  return np.array(inputData), np.array(targetData)
  

In [0]:
inputTrain, outputTrain = generateSeqs(train, train.Close, timeSteps)
inputTest, outputTest = generateSeqs(test, test.Close, timeSteps)

print(inputTrain.shape, outputTrain.shape)

(203, 20, 12) (203,)

In [0]:
model = keras.Sequential()
model.add(
  keras.layers.Bidirectional(
    keras.layers.LSTM(
      units=lstmUnits, 
      input_shape=(inputTrain.shape[1], inputTrain.shape[2])
    )
  )
)
model.add(keras.layers.Dropout(rate=dropoutRate))
model.add(keras.layers.Dense(units=1))
model.compile(loss=modelLoss, optimizer=modelOptimizer)

In [0]:
model.fit(
    inputTrain, outputTrain, 
    epochs=numEpochs, 
    validation_split=0.1,
    shuffle=False
)

Epoch 1/100
WARNING:tensorflow:Layer bidirectional is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2. The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

1/6 [====>.........................] - ETA: 0s - loss: 0.95284/6 [===================>..........] - ETA: 0s - loss: 0.38406/6 [==============================] - 1s 163ms/step - loss: 0.2758 - val_loss: 0.0290
Epoch 2/100
1/6 [====>.........................] - ETA: 0s - loss: 0.42614/6 [===================>..........] - ETA: 0s - loss: 0.30366/6 [==============================] - 0s 26ms/step - loss: 0.2155 - val_loss: 0.0076
Epoch 3/100
1/6 [====>.........................] - ETA: 0s - loss: 0.06084/6 [===================>..........] - ETA: 0s - loss: 0.05076/6 [==============================] - 0s 24ms/step - loss: 0.0461 - val_loss: 0.0199
Epoch 4/100
1/6 [====>.........................] - ETA: 0s - loss: 0.16554/6 [===================>..........] - ETA: 0s - loss: 0.08796/6 [==============================] - ETA: 0s - loss: 0.06496/6 [==============================] - 0s 28ms/step - loss: 0.0649 - val_loss: 0.0047
Epoch 5/100
1/6 [====>.........................] - ETA: 0s - loss: 0.08073/6 [==============>...............] - ETA: 0s - loss: 0.07686/6 [==============================] - ETA: 0s - loss: 0.04956/6 [==============================] - 0s 27ms/step - loss: 0.0495 - val_loss: 0.0022
Epoch 6/100
1/6 [====>.........................] - ETA: 0s - loss: 0.01704/6 [===================>..........] - ETA: 0s - loss: 0.04916/6 [==============================] - 0s 25ms/step - loss: 0.0361 - val_loss: 0.0040
Epoch 7/100
1/6 [====>.........................] - ETA: 0s - loss: 0.02404/6 [===================>..........] - ETA: 0s - loss: 0.04566/6 [==============================] - 0s 24ms/step - loss: 0.0335 - val_loss: 0.0012
Epoch 8/100
1/6 [====>.........................] - ETA: 0s - loss: 0.02244/6 [===================>..........] - ETA: 0s - loss: 0.03756/6 [==============================] - 0s 24ms/step - loss: 0.0274 - val_loss: 0.0014
Epoch 9/100
1/6 [====>.........................] - ETA: 0s - loss: 0.03684/6 [===================>..........] - ETA: 0s - loss: 0.03786/6 [=============

In [0]:
predictTrain = closeScaler.inverse_transform(model.predict(inputTrain)).ravel().tolist()
actualTrain = closeScaler.inverse_transform(outputTrain.reshape(-1,1)).ravel().tolist()
trainPredictionsAndActual = sc.parallelize(list(zip(predictTrain, actualTrain)))

predictTest = closeScaler.inverse_transform(model.predict(inputTest)).ravel().tolist()
actualTest = closeScaler.inverse_transform(outputTest.reshape(-1,1)).ravel().tolist()
testPredictionsAndActual = sc.parallelize(list(zip(predictTest, actualTest)))

In [0]:
trainMetrics = RegressionMetrics(trainPredictionsAndActual)
print("Training RMSE = %s" % trainMetrics.rootMeanSquaredError)

testMetrics = RegressionMetrics(testPredictionsAndActual)
print("Test RMSE = %s" % testMetrics.rootMeanSquaredError)

Training RMSE = 5.278486723487297
Test RMSE = 10.405613891742997